In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arc-prize-2024/arc-agi_training_solutions.json
/kaggle/input/arc-prize-2024/arc-agi_evaluation_solutions.json
/kaggle/input/arc-prize-2024/arc-agi_evaluation_challenges.json
/kaggle/input/arc-prize-2024/sample_submission.json
/kaggle/input/arc-prize-2024/arc-agi_training_challenges.json
/kaggle/input/arc-prize-2024/arc-agi_test_challenges.json


In [2]:
import json

training_solutions_path = '/kaggle/input/arc-prize-2024/arc-agi_training_solutions.json'
evaluation_solutions_path = '/kaggle/input/arc-prize-2024/arc-agi_evaluation_solutions.json'
evaluation_challenges_path = '/kaggle/input/arc-prize-2024/arc-agi_evaluation_challenges.json'
sample_submission_path = '/kaggle/input/arc-prize-2024/sample_submission.json'
training_challenges_path = '/kaggle/input/arc-prize-2024/arc-agi_training_challenges.json'
test_challenges_path = '/kaggle/input/arc-prize-2024/arc-agi_test_challenges.json'

#function to load JSON data
def load_json_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

#load each dataset
training_solutions = load_json_data(training_solutions_path)
evaluation_solutions = load_json_data(evaluation_solutions_path)
evaluation_challenges = load_json_data(evaluation_challenges_path)
sample_submission = load_json_data(sample_submission_path)
training_challenges = load_json_data(training_challenges_path)
test_challenges = load_json_data(test_challenges_path)

In [3]:
#inspect the structure of the data
def inspect_data(data, name):
    print(f"Inspecting {name}:")
    if isinstance(data, list):
        print(f"Number of items: {len(data)}")
        if len(data) > 0:
            print(f"Example item: {data[0]}")
    elif isinstance(data, dict):
        print(f"Number of keys: {len(data.keys())}")
        if len(data.keys()) > 0:
            first_key = list(data.keys())[0]
            print(f"Example item under key '{first_key}': {data[first_key]}")
    else:
        print("Unknown data type")
    print("\n")

inspect_data(training_solutions, "Training Solutions")
inspect_data(evaluation_solutions, "Evaluation Solutions")
inspect_data(evaluation_challenges, "Evaluation Challenges")
inspect_data(sample_submission, "Sample Submission")
inspect_data(training_challenges, "Training Challenges")
inspect_data(test_challenges, "Test Challenges")

Inspecting Training Solutions:
Number of keys: 400
Example item under key '007bbfb7': [[[7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 7, 0, 0, 0, 0, 7, 7, 0], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 7, 0, 0, 0, 0, 7, 7, 0], [7, 0, 7, 7, 0, 7, 0, 0, 0], [7, 0, 7, 7, 0, 7, 0, 0, 0], [7, 7, 0, 7, 7, 0, 0, 0, 0]]]


Inspecting Evaluation Solutions:
Number of keys: 400
Example item under key '00576224': [[[3, 2, 3, 2, 3, 2], [7, 8, 7, 8, 7, 8], [2, 3, 2, 3, 2, 3], [8, 7, 8, 7, 8, 7], [3, 2, 3, 2, 3, 2], [7, 8, 7, 8, 7, 8]]]


Inspecting Evaluation Challenges:
Number of keys: 400
Example item under key '00576224': {'test': [{'input': [[3, 2], [7, 8]]}], 'train': [{'input': [[8, 6], [6, 4]], 'output': [[8, 6, 8, 6, 8, 6], [6, 4, 6, 4, 6, 4], [6, 8, 6, 8, 6, 8], [4, 6, 4, 6, 4, 6], [8, 6, 8, 6, 8, 6], [6, 4, 6, 4, 6, 4]]}, {'input': [[7, 9], [4, 3]], 'output': [[7, 9, 7, 9, 7, 9], [4, 3, 4, 3, 4, 3], [9, 7, 9, 7, 9, 7], [3, 4, 3, 4, 3, 4], [7, 9, 7, 9, 7, 

In [4]:
def get_max_grid_size(challenges, solutions):
    max_input_height, max_input_width = 0, 0
    max_output_height, max_output_width = 0, 0
    
    for key in challenges.keys():
        challenge = challenges[key]
        for example in challenge['train']:
            input_grid = example['input']
            output_grid = example['output']
            max_input_height = max(max_input_height, len(input_grid))
            max_input_width = max(max_input_width, len(input_grid[0]))
            max_output_height = max(max_output_height, len(output_grid))
            max_output_width = max(max_output_width, len(output_grid[0]))
        for test_case in challenge['test']:
            test_input = test_case['input']
            max_input_height = max(max_input_height, len(test_input))
            max_input_width = max(max_input_width, len(test_input[0]))
            #assuming test_output size can be derived similarly

    return max_input_height, max_input_width, max_output_height, max_output_width

max_input_height, max_input_width, max_output_height, max_output_width = get_max_grid_size(training_challenges, training_solutions)
print(f"Max input size: {max_input_height}x{max_input_width}, Max output size: {max_output_height}x{max_output_width}")

Max input size: 30x30, Max output size: 30x30
